In [1]:
from bs4 import BeautifulSoup
import requests
import time
import os

In [2]:
def download_and_save_html(urls, interval, dir_name):
    """
    urlsに入っているurlからhtmlファイルをダウンロードし、
    page1.html, page2.html, ...としてディレクトリdir_nameの中に保存する
    intervalはダウンロード間隔
    このダウンロードには少なくともlen(urls) * interval秒かかる
    """
    for n, url in enumerate(urls, start=1):
        time.sleep(interval)  # サーバーに負荷をかけないように時間間隔を設ける
        text = requests.get(url).text
        file_name = 'page{}.html'.format(n)
        path = os.path.join(dir_name, file_name)
        with open(path, 'w') as f:
            f.write(text)

In [3]:
def get_recipe_urls(path_to_html):
    """
    cookpadの検索結果ページのhtmlから
    各レシピのurlのリストを返す
    """
    with open(path_to_html, 'r') as f:
        soup = BeautifulSoup(f)
    tags_a = soup.find_all('a', class_='recipe-title font13 ')
    urls = ['https://cookpad.com' + tag_a.get('href') for tag_a in tags_a]
    return urls

In [4]:
# ダウンロードしたすべて検索結果からレシピのページへのurlを
# 取得し、recipe_urlsにまとめる
n_search_results = 2
paths = ['search_results/page{}.html'.format(n) \
         for n in range(1, n_search_results+1)]
recipe_urls = []
for path in paths:
    recipe_urls.extend(get_recipe_urls(path))

In [5]:
# 検索結果ページに含まれるレシピの件数は10のため、
# レシピの総件数はn_search_results*10となる
assert len(recipe_urls) == n_search_results*10

In [6]:
# レシピのhtmlを保存するディレクトリを作る
os.makedirs('recipes', exist_ok=True)

# すべてのレシピのhtmlをダウンロードする。
# たとえば、n_search_results = 500なら、
# n_search_results*10 = 5000秒　= 84分ほどかかる。
download_and_save_html(urls=recipe_urls, interval=1, dir_name='recipes')